# Scalable Bayesian Modeling

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/symeneses/SBM/blob/main/notebooks/template.ipynb)

In [ ]:
# Execute only if executing in Google Colab
# !rm -r ./src
# !rm -r ./data
# !git clone https://github.com/symeneses/SBM
# !mv ./SBM/src ./src
# !mv ./SBM/data ./data
# !rm -r ./SBM
# !pip install --upgrade pip
# !pip install --upgrade numpyro==0.11.0 pymc==5.0.2 blackjax==0.9.6 seaborn

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

cores = os.cpu_count()
os.environ["XLA_FLAGS"] = f'--xla_force_host_platform_device_count={cores}'
root_path = os.path.abspath(os.pardir)
if root_path not in sys.path:
    sys.path.append(root_path)

## Importing the needed libraries

In [ ]:
import pandas as pd
import numpy as np
import numpyro
import pymc as pm

from src.data import data_generator
from src.handler import Handler
from src.diagnostics import convergency_validator, dist_validator
from src.plots import plot_ess_ps, plot_monitor

pd.set_option('display.precision', 2)
RANDOM_SEED = 8957


## Getting the data

Create a Pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) with the variables to be used to define the model.

✍🏽 User input required.

In [ ]:
# Code to generate the data, name the final results as data
data = pd.DataFrame()

## Defining the model

In the following cells, write the code for the models you want to compare. To make comparing the models and results easier, use the same name of variables in each version of the model.

**For models in PyMC:** Create an annotated function, as in the example, that returns a `pm.model.Model`.

**For models in Numpyro:** Use the same variable names of the `DataFrame` `data` for the arguments of the function model. These arguments should contain only columns present in `data`.

✍🏽 User input required.

In [ ]:

def pymc_model_gen(data: pd.DataFrame) -> pm.model.Model:

    with pm.Model() as pymc_model:
        # Your model here
        mu_alpha = pm.Normal("μ_α", mu=0.0, sigma=1.0)
    return pymc_model

In [ ]:
import numpyro.distributions as dist
from numpyro.infer import NUTS


def model():
    # Your model here
    return

numpyro_kernel = NUTS(model)

## Inference
Here, we will create create valid [InferenceData](https://python.arviz.org/en/latest/api/generated/arviz.InferenceData.html) objects and a set of metrics to measure performance for the selected models, data sizes and samplers.

In [ ]:
data.shape

### Creating multiple data sizes

To have a benchmark using different data sizes, we will use the function `data_generator` which will use the original dataset to generate datasets with the given `sizes` or filtering the original data in the parameter `filters`. You can also add Gaussian noise to selected variables using the parameter `include_noise`.

✍🏽 User input required.

In [ ]:
data_sizes = []
include_noise = []
datasets = data_generator(data, include_noise=include_noise, sizes=data_sizes)
# If you want to work only with the original data, use this instead
# datasets = {"original_size": data}

### Sampling

⚠️ You should at least use 2 chains to be able to calculate correctly the diagnostics.

✍🏽 User input required.

In [ ]:
# Use the following path if working in Google Colab
# output_path = "data/results"
output_path = "../data/results"
models = {"pymc": pymc_model_gen, "numpyro": numpyro_kernel}
pymc_samplers = ["default", "numpyro", "blackjax"]
draws = 2000
tune = 2000
# It's recommended to use between 2 and 4 chains
chains = 2

# sampling all models
handler = Handler(models, datasets, pymc_samplers, output_path)
infer_data, results = handler.execute(draws, tune, chains, RANDOM_SEED)

## Check Convergency

After sampling, the function `convergency_validator` will help you know if the models have converged. This function use the [rank normalized splitR-hat](https://python.arviz.org/en/latest/api/generated/arviz.rhat.html).

An $\hat R$ > 1.05 indicates convergence failures. In this case, the results of the next step `Validate Results` **can't be considered** as they assumed the MCMC has converged. 

In [ ]:
convergency_validator(infer_data)

## Validate results 

To check that the models are sampling from the same distributions. The function `dist_validator` will estimate the ranges of the mean for each variable using the [MCSE](https://python.arviz.org/en/latest/api/generated/arviz.mcse.html) of one of the models as reference. You can either give a `seed` to choose a model randomly or give the key `ref_key` of a selected model. The reference model will be compared only with others using the same sample size.

The percentages displayed indicate how many variables are within the calculated range using ±3 sigma. The values should be in theory greater or equal to `95%` following a weaker [three-sigma rule](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule).

In [ ]:
results_val, summaries = dist_validator(infer_data, ref_key="pymc_default_1549")

## Comparison

### Resources metrics

In [ ]:
plot_monitor(results)

### ESS

To determine the sampler performance, we use the **Effective Sample Size** ([ESS](https://python.arviz.org/en/latest/api/generated/arviz.ess.html])) calculated per second.

In [ ]:
plot_ess_ps(results, summaries, data_sizes=data_sizes)

In [ ]:
plot_ess_ps(results, summaries, data_sizes=[max(data_sizes)])

Let's rank the best across all data sizes:

In [ ]:
summary = results.drop(columns=["current_memory", "peak_memory"])
summary["rank"] = summary.sort_values(["size", "ess_mean/s"], ascending=[True, False]) \
            .groupby(['size']) \
            .cumcount() + 1
summary.sort_values(["size", "rank"])

Let's see the best options in each data size:

In [ ]:
summary.query("rank == 1").sort_values("size")